In [1]:
import numpy as np
import pandas as pd
import pickle
import random
from sklearn.feature_extraction.text import *

import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.cross_validation import train_test_split

/root/anaconda2/envs/nlp/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pickle.load(open("/root/Objects/Fulldata_wY_500", "rb"))

In [4]:
train_data = data.loc[data.Date < '2009-01-01', :]
dev_data = data.loc[(data.Date >= '2009-01-01') & (data.Date <= '2010-12-31'), :]
test_data = data.loc[data.Date >= '2011-01-01', :]

train_label = train_data.label
dev_label = dev_data.label
test_label = test_data.label

#train_text_features.shape
train_data['Surprise'].fillna(0, inplace=True)
dev_data['Surprise'].fillna(0, inplace=True)
#train_data.fillna(0, inplace=True)
pd.options.mode.chained_assignment = None

/root/anaconda2/envs/nlp/lib/python2.7/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [5]:
vectorizer = CountVectorizer(min_df=1, stop_words='english')
train_text_features = vectorizer.fit_transform(train_data.bow)

dev_text_features = vectorizer.transform(dev_data.bow)


################Combine Text Feature with Earning Surprise ################################
all_data_train = np.append(train_text_features.toarray(),\
                           np.array(train_data.Surprise).reshape(len(train_data.Surprise),1), 1)
all_data_dev = np.append(dev_text_features.toarray(),\
                         np.array(dev_data.Surprise).reshape(len(dev_data.Surprise),1), 1)

##  Sample smaller subset of  dev dataset

In [7]:
import random

#randomly drawn negative cases from negative case dataset
randomw_draw = random.sample(range(0, dev_data.shape[0]), 5000) #11000
all_data2 = dev_data.iloc[randomw_draw,:]

### Create Y label  

In [6]:
def create_label(data_in):
#Generate new labels
  label_y = []
  for sign in data_in:
    if sign == 'DOWN':
      label_y.append([0, 0, 1])
    elif sign == 'STAY':
      label_y.append([0, 1, 0])
    else:
      label_y.append([1, 0, 0])
  
  label_y = np.asarray(label_y)
  return(label_y)

In [7]:
y_train = create_label(train_data.label)
y_dev = create_label(dev_data.label)

##  TensforFlow Code for  Feed Forward neural networks

In [12]:
def init_weights(shape):
    """
    Initialized Weights
    """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward Propagation.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat


# Layer's sizes
x_size = all_data_train.shape[1]   # Number of inputs
h_size = 200   #256                # Hidden nodes
y_size = 3   # Number of class

# Input/Output
X = tf.placeholder("float", shape=[None, x_size])
y = tf.placeholder("float", shape=[None, y_size])

# Weight initializations
w1 = init_weights((x_size, h_size))
w2 = init_weights((h_size, y_size))

# Forward propagation
y_hat    = forwardprop(X, w1, w2)
predict = tf.argmax(yhat, dimension=1)

# Backward propagation
cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_hat, y))
updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

# Run SGD
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

for epoch in range(1000):
    
    #Randomly Draw a sample of data for training each epoch
    randomw_draw = random.sample(range(0, all_data_train.shape[0]), 5000) #11000
    all_data2 = all_data_train[randomw_draw,:]
    y_train2 = y_train[randomw_draw,:]
    # Train with each example
   
    sess.run([updates], feed_dict={X: all_data2, y: y_train2})

    train_accuracy = np.mean(np.argmax(y_train2, axis=1) ==
                                 sess.run(predict, feed_dict={X: all_data2, y: y_train2}))
    test_accuracy  = np.mean(np.argmax(y_dev, axis=1) ==
                                 sess.run(predict, feed_dict={X: all_data_dev, y: y_dev}))
    #train_accuracy = 0
    if (epoch +1) % 100 == 0:
      print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%" % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

sess.close()

Epoch = 100, train accuracy = 45.86%, test accuracy = 46.62%
Epoch = 200, train accuracy = 45.92%, test accuracy = 47.33%
Epoch = 300, train accuracy = 48.04%, test accuracy = 47.83%
Epoch = 400, train accuracy = 48.38%, test accuracy = 48.02%
Epoch = 500, train accuracy = 48.70%, test accuracy = 48.13%
Epoch = 600, train accuracy = 49.16%, test accuracy = 48.40%
Epoch = 700, train accuracy = 50.52%, test accuracy = 48.61%
Epoch = 800, train accuracy = 50.46%, test accuracy = 48.69%
Epoch = 900, train accuracy = 50.08%, test accuracy = 48.91%
Epoch = 1000, train accuracy = 51.78%, test accuracy = 48.95%


## Best Result 

In [ ]:
#RANDOM_SEED = 42
#tf.set_random_seed(RANDOM_SEED)

def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat


# Layer's sizes
x_size = all_data_train.shape[1]   # Number of input nodes: 4 features and 1 bias
h_size = 256                # Number of hidden nodes
y_size = 3   # Number of outcomes (3 iris flowers)

# Symbols
X = tf.placeholder("float", shape=[None, x_size])
y = tf.placeholder("float", shape=[None, y_size])

# Weight initializations
w1 = init_weights((x_size, h_size))
w2 = init_weights((h_size, y_size))

# Forward propagation
yhat    = forwardprop(X, w1, w2)
predict = tf.argmax(yhat, dimension=1)

# Backward propagation
cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(yhat, y))
updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

# Run SGD
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

for epoch in range(1500):
  
    randomw_draw = random.sample(range(0, all_data_train.shape[0]), 5000) #11000
    all_data2 = all_data_train[randomw_draw,:]
    y_train2 = y_train[randomw_draw,:]
    # Train with each example
   
    sess.run([updates], feed_dict={X: all_data2, y: y_train2})

    #train_accuracy = np.mean(np.argmax(y_train2, axis=1) ==
    #                             sess.run(predict, feed_dict={X: all_data2, y: y_train2}))
    test_accuracy  = np.mean(np.argmax(y_dev, axis=1) ==
                                 sess.run(predict, feed_dict={X: all_data_dev, y: y_dev}))
    
    #Train Accuracy set to zero to speed up the process
    train_accuracy = 0
    if (epoch +1) % 50 == 0:
      print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%" % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

sess.close()

Epoch = 50, train accuracy = 0.00%, test accuracy = 46.36%
Epoch = 100, train accuracy = 0.00%, test accuracy = 47.09%
Epoch = 150, train accuracy = 0.00%, test accuracy = 47.40%
Epoch = 200, train accuracy = 0.00%, test accuracy = 47.82%
Epoch = 250, train accuracy = 0.00%, test accuracy = 47.95%
Epoch = 300, train accuracy = 0.00%, test accuracy = 48.15%
Epoch = 350, train accuracy = 0.00%, test accuracy = 48.15%
Epoch = 400, train accuracy = 0.00%, test accuracy = 48.22%
Epoch = 450, train accuracy = 0.00%, test accuracy = 48.26%
Epoch = 500, train accuracy = 0.00%, test accuracy = 48.42%
Epoch = 550, train accuracy = 0.00%, test accuracy = 48.50%
Epoch = 600, train accuracy = 0.00%, test accuracy = 48.72%
Epoch = 650, train accuracy = 0.00%, test accuracy = 48.90%
Epoch = 700, train accuracy = 0.00%, test accuracy = 48.94%
Epoch = 750, train accuracy = 0.00%, test accuracy = 48.97%
Epoch = 800, train accuracy = 0.00%, test accuracy = 49.19%
Epoch = 850, train accuracy = 0.00%, test